## 1. Qubit Rotation (Construção de função de rotação de um Qubit)

> A relação dessa aula introdutória, vem de que o ML é um aproximador universal de funções, então na prática isso evidencia como pode-se usar circuitos quânticos (essencialmente funções) para atingir valores desejados.

In [ ]:
!pip install -q pennylane
!pip install -q jax==0.6.2 jaxlib==0.6.2
!pip install jaxopt

In [1]:
import pennylane as qml
from jax import numpy as np
import jaxopt
import jax

/usr/local/lib/python3.12/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.7.2 is installed, but it is not compatible with the installed jaxlib version 0.6.2, so it will not be used.
  warnings.warn(


#### Entendendo: `Quantum Circuit`, `Device` e `Qnode`.

- *Device* : Objeto computacional que consegue realizar as operações quânticas e retornar valores de medição.
	- Parâmetro 1: `name` para uso de *qubits* costuma ser o "lightning.qubit"
	- Parâmetro 2: `wires` determina a quantidade de *qubits* inicializadas com o *device*

- *Quantum Circuit* : Função quântica responsável por realizar operações nos *qubits*.
	- Precisa ter operadores quânticos e apenas um operador por linha, em ordem de aplicação.
	- Precisa retornar um ou vários valores de medição (binários clássicos).

- *QNode* : Uma abstração que recebe uma função quântica (o sistema que realiza operações em um ou mais *qubits* - **quantum circuit**)  e consegue executar  e avaliar os valores desse circuito no *device*, basicamente o link entre o circuito e o **hardware quântico** (*device*).
	- Construídos pela `Qnode class` ou  pelo `qnoded() decorator` passando como argumento o *device*.

In [2]:
dev1 = qml.device("lightning.qubit", wires=1)

In [3]:
@qml.qnode(dev1)
def circuit(params):

  # Ângulo de rotação, qubit alvo
  qml.RX(params[0], wires=0)
  qml.RY(params[1], wires=0)

  # Valor binário da medição na base computacional
  return qml.expval(qml.PauliZ(0))

In [4]:
params = np.array([0.54, 0.12])
print(circuit(params))

0.85154057


#### Calculando *Quantum Gradients*

- Quantum Gradient* : O gradiente do circuito é calculado pelo mesmo *device* e pode ser diferenciado utilizando a função `jax.grad()`:
	- Retorna outra função que representa o gradiente do circuito.
	- Para obter o resultado desse gradiente, basta chamar a função retornada com os argumentos utilizados no circuito, retornando um array com dimensão igual a quantidade de argumentos.

In [5]:
# O circuito para realizar o gradiente, e quantos parâmetros vai ter a função de obter o valor do gradiente
gradient_circuit = jax.grad(circuit, argnums=0)

In [ ]:
# Recebeu apenas uma lista (com dois elementos) como argumento e retorna um array bi-dimensional
print(dcircuit(params))

#### Otimização

O próprio PennyLane oferece uma variedade de otimizadores para o gradiente descendente.  Esses otimizadores (clássicos) recebem uma função de custo e parâmetros inicias, e o processo automático do PennyLane realiza os cálculos e as épocas.

- A função de custo varia a depender do resultado que se deseja obter, para um resultado que pertença aos dois possíveis valores do circuito, a função de custo pode ser o próprio retorno do circuito.
- Utilizando o otimizador com esse valor da função de custo, em um laço que realiza uma quantidade de épocas (etapas de aprendizado) arbitrárias conseguimos aproximar o valor da função do circuito para o valor esperado.

In [7]:
# Otimizador para valor esperado -1

def cost(x):
  return circuit(x)

In [11]:
init_params = np.array([0.011, 0.012])
print(cost(init_params))

0.9998675


In [14]:
# Otimizador (Clássico) -> Função de custo, "Learning Rate", Técnicas para convergência mais rápida (irrelevamente no momento)
opt = jaxopt.GradientDescent(cost, stepsize=0.4, acceleration = False)

# Número de épocas (etapas de "aprendizado")
steps = 100
params = init_params
opt_state = opt.init_state(params)

for i in range(steps):

    # Atualiza os paramêtros e os estados atual, o objetivo é que o estado chegue no valor desejado ou muito próximo disso, e saber quais paramêtros permitem isso
    params, opt_state = opt.update(params, opt_state)

    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))

print("Optimized rotation angles: {}".format(params))

Cost after step     5:  0.9961779
Cost after step    10:  0.8974943
Cost after step    15:  0.1440490
Cost after step    20: -0.1536721
Cost after step    25: -0.9152496
Cost after step    30: -0.9994046
Cost after step    35: -0.9999964
Cost after step    40: -1.0000000
Cost after step    45: -1.0000000
Cost after step    50: -1.0000000
Cost after step    55: -1.0000000
Cost after step    60: -1.0000000
Cost after step    65: -1.0000000
Cost after step    70: -1.0000000
Cost after step    75: -1.0000000
Cost after step    80: -1.0000000
Cost after step    85: -1.0000000
Cost after step    90: -1.0000000
Cost after step    95: -1.0000000
Cost after step   100: -1.0000000
Optimized rotation angles: [7.1526556e-18 3.1415925e+00]


---


## 2. Quantum gradients with backpropagation

> Aqui é feito uma comparação entre técnicas para calcular gradientes quânticas:   A regra de deslocamento dos parâmetros que trata o sistema como uma "caixa preta" e o Backpropagation que vem do clássico e necessita de maior observabilidade.


In [ ]:
!pip install -q matplotlib

In [ ]:
import pennylane as qml
from jax import numpy as jnp
from matplotlib import pyplot as plt
import jax

#### *The parameter-shif Rule* (Regra de deslocamento de parâmetros)

Esse método é mais útil para lidar com o cálculo de gradientes em Computação Quântica Variacional (QMV), pois não precisa acessar estados internos do circuito, o que nos casos de hardwares quânticos não é possível.

- A derivada é calculada executando a função de custo quântica várias vezes (para obter duas avaliações de expectância confiáveis), com o deslocamento dos parâmetros de entrada em um valor fixo (aumentando e diminuindo).

- O gradiente (que é o vetor de derivadas) é obtido por uma diferença finita e exata entre essas duas medições de Expectância (Custo).

In [15]:
#  Implementação de um Circuito Quântico Variacional

dev = qml.device("default.qubit", wires=3)

def CNOT_ring(wires):
    """Apply CNOTs in a ring pattern"""
    n_wires = len(wires)

    for w in wires:
        qml.CNOT([w % n_wires, (w + 1) % n_wires])


# Já é definido qual o método do gradiente será utilizado
@qml.qnode(dev, diff_method="parameter-shift")
def circuit(params):
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=1)
    qml.RZ(params[2], wires=2)

    CNOT_ring(wires=[0, 1, 2])

    qml.RX(params[3], wires=0)
    qml.RY(params[4], wires=1)
    qml.RZ(params[5], wires=2)

    CNOT_ring(wires=[0, 1, 2])
    return qml.expval(qml.PauliY(0) @ qml.PauliZ(2))


In [16]:
# Resultado com parâmetros aleatórios

jax.config.update("jax_platform_name", "cpu")
jax.config.update('jax_enable_x64', True)
key = jax.random.PRNGKey(42)

params = jax.random.normal(key, [6])
print("Parameters:", params)
print("Expectation value:", circuit(params))

Parameters: [-0.18471175 -2.16982456  0.18693555  0.61226536  0.48962495  0.3690043 ]
Expectation value: -0.324069434254729


In [ ]:
# Função para aplicar a regra de deslocamento no circuito variacional

def parameter_shift_term(qnode, params, i):
    shifted = params.copy()
    shifted = shifted.at[i].add(jnp.pi/2)
    forward = qnode(shifted)  # Valor de expectância obtido com aumento

    shifted = shifted.at[i].add(-jnp.pi)
    backward = qnode(shifted) # Valor de expectância obtido com diminuição


    return 0.5 * (forward - backward)

# Gradiente relacionado ao primeiro parâmetro
print(parameter_shift_term(circuit, params, 0))

- **Problemática**: Essa regra possui uma dificuldade em escalabilidade:

	- Para cada parâmetro é preciso estimar o circuito 2 vezes, com o acréscimo e decréscimo do valor fixo no parâmetro, isso sem contar com a necessidade de realizar essa mesma medição inúmeras vezes para garantir um valor de expectância seguro.

	- Além disso, quanto mais complexo o circuito  a complexidade também cresce exponencialmente (maior quantidade de *qubits*, portas, emaranhamento...)

In [ ]:
# LOOP: Para obter o gradiente de todos os parâmetros (nesse caso são poucos parâmetros)

def parameter_shift(qnode, params):
    gradients = jnp.zeros([len(params)])

    for i in range(len(params)):
        gradients = gradients.at[i].set(parameter_shift_term(qnode, params, i))

    return gradients

print(parameter_shift(circuit, params))

#### *Backpropagation*

Esse método utilizado na computação clássica, possui uma eficiência computacional muito superior ao método anterior, exigindo apenas uma passagem pela execução do circuito para obter o gradiente de todos os parâmetros.

	- De forma resumida: são feitos cálculos intermediários para cada operação (porta) e armazenados na memória, para que em seguida esses resultados sejam percorridos no sentido inverso e regra da cadeia aplicada repetidamente retorna os gradientes (derivadas parciais), assim, precisa de apenas um único passo na função.

	- Contudo, no caso quântico percorrer o circuito (função) armazenar seus valores e depois percorrer no sentido inverso e computar novos valores, é algo que o hardware quântico não consegue realizar, sem problemas de eficiência ou colapso dos *qubits*.

In [17]:
# Device feito para aceitar o uso do brackpopagation no caso quântico
dev = qml.device("default.qubit", wires=4)

@qml.qnode(dev, diff_method="backprop")
def circuit(params):
    qml.StronglyEntanglingLayers(params, wires=[0, 1, 2, 3])
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1) @ qml.PauliZ(2) @ qml.PauliZ(3))

# Inicializando
param_shape = qml.StronglyEntanglingLayers.shape(n_wires=4, n_layers=15)
params = jax.random.normal(key, param_shape) * 0.1

print(circuit(params))

0.9023492424085713
